<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/text_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download ru_core_news_sm
!python -m spacy download ru_core_news_md

2021-04-05 20:31:31.434215: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 17.9 MB 1.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
2021-04-05 20:31:41.310406: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 43.9 MB 16 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [2]:
pip install pymorphy2[fast]

In [8]:
pip install Gensim

In [58]:
import spacy
import gensim
from gensim.models import Word2Vec 
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np

In [4]:
nlp = spacy.load('ru_core_news_md')

In [5]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_ru.txt

--2021-04-05 20:32:00--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_ru.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3638613 (3.5M) [text/plain]
Saving to: ‘neuroleptic_ru.txt.1’

neuroleptic_ru.txt. 100%[===================>]   3.47M  21.4MB/s    in 0.2s    

2021-04-05 20:32:01 (21.4 MB/s) - ‘neuroleptic_ru.txt.1’ saved [3638613/3638613]



In [6]:
!wget https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz

--2021-04-05 20:32:01--  https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140310878 (134M) [application/x-gzip]
Saving to: ‘news_upos_cbow_300_2_2017.bin.gz.1’

news_upos_cbow_300_ 100%[===================>] 133.81M  28.1MB/s    in 5.5s    

2021-04-05 20:32:07 (24.4 MB/s) - ‘news_upos_cbow_300_2_2017.bin.gz.1’ saved [140310878/140310878]



In [28]:
handler = open('neuroleptic_ru.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()

In [29]:
posts = str_list = my_txt.split('----')

In [30]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]
sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]

In [17]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('news_upos_cbow_300_2_2017.bin.gz', binary=True)

In [34]:
model = Word2Vec(size=300, min_count=1)
model.build_vocab(sentences_tokenized)
total_examples = model.corpus_count
model.build_vocab([list(w2v.vocab.keys())], update=True)
model.intersect_word2vec_format("news_upos_cbow_300_2_2017.bin.gz", binary=True)
model.train(sentences_tokenized, total_examples=total_examples, epochs=model.iter)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


(1321032, 1530870)

In [36]:
model.most_similar('препарат')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('ну', 0.9999090433120728),
 ('которые', 0.9998809099197388),
 ('всего', 0.999872088432312),
 ('другие', 0.999860942363739),
 ('симптомы', 0.9998470544815063),
 ('антидепрессанты', 0.999845564365387),
 ('трудно', 0.9998406171798706),
 ('назначение', 0.9998394250869751),
 ('эту', 0.9998353123664856),
 ('спать', 0.9998342990875244)]

In [71]:
# model['алимемазин']
model.wv.most_similar('алимемазин')

[('ибо', 0.9970612525939941),
 ('является', 0.9970073103904724),
 ('цитата', 0.9970026612281799),
 ('со', 0.9969857335090637),
 ('хороший', 0.996950089931488),
 ('свою', 0.9969457387924194),
 ('дофамина', 0.9969419240951538),
 ('зрения', 0.9969398975372314),
 ('под', 0.9969329833984375),
 ('действие', 0.9968994855880737)]

In [68]:
cosine_similarity(np.atleast_2d(model.wv.get_vector('алимемазин')), \
                  np.atleast_2d(model.wv.get_vector('азалептин')))[0][0]

0.9956967

In [82]:
cosine_similarity(np.atleast_2d(model.wv.get_vector('алимемазин')), \
                  np.atleast_2d(model.wv.get_vector('стул')))[0][0]

0.7902447

In [48]:
posts[9]

'\nПринимал порядка 2-3 недель от 10 до 20 мг Алимемазина, совместно с 200 мг Азалептина и 30 мг Ремерона.Всё это принималось за полчаса до сна.Лично я не почувствовал действие Тералиджена, словно и не принимал его вовсе.Мою довольно средневзвешенную ипохондрию он не купировал,равно как и депрессивные симптомы.\n'

**Нужно ли мне сначала извлекать названия лекарств?**

In [41]:
posts[0]

'На рынок снова вернулся алимемазин (раньше терален назывался). Хотелось бы узнать Ваше мнение. Кто принимал в своей практике, что хорошего, что плохого\n\n____________________________________________\nСообщение отредактировано.\n\nДобавлена ссылка на ролик в You Tube.\n\nСообщение отредактировал Гилев: 12 Апрель 2020 - 07:55\nДобавлена ссылка на ролик в You Tube\n'

In [42]:
doc = nlp(u'Кто принимал его в своей практике? Это не хороший препарат.')

In [43]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [44]:
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_, 
            [child for child in token.children], morph.parse(token.text)[0].tag)


Кто nsubj принимал VERB [] NPRO,masc sing,nomn
принимал ROOT принимал VERB [Кто, его, практике, ?] VERB,impf,tran masc,sing,past,indc
его obj принимал VERB [] NPRO,masc,3per,Anph sing,accs
в case практике NOUN [] PREP
своей det практике NOUN [] ADJF,Apro,Anph femn,sing,gent
практике obl принимал VERB [в, своей] NOUN,inan,femn sing,loct
? punct принимал VERB [] PNCT
Это nsubj препарат NOUN [] PRCL
не advmod хороший ADJ [] PRCL
хороший amod препарат NOUN [не] ADJF,Qual masc,sing,nomn
препарат ROOT препарат NOUN [Это, хороший, .] NOUN,inan,masc sing,nomn
. punct препарат NOUN [] PNCT


In [ ]:
def find_args(my_text, my_spacy):
  sents = text.split('.')
  for sent in sents:
  doc = my_spacy(u'Кто принимал его в своей практике')

In [22]:
token_list = [i for i in doc]

In [18]:
for token in doc:
  if token.dep_ == 'obj':
    print(token.text, token.dep_, token.head.text, token.head.pos_,
        [child for child in token.children])

его obj принимал VERB []


In [6]:
str_list[1]

'\nОрто сказал(а) 27.11.2008, 19:59:\nНа рынок снова вернулся алимемазин (раньше терален назывался). Хотелось бы узнать Ваше мнение. Кто принимал в своей практике, что хорошего, что плохого\n\nЭто тот редкий случай , когда антипсихотик оставил самые приятные впечатления ( к сожалению про моно-терапию рассказать не могу т.к принимал его совместно с антидепрессантами )\nОрто , а где его можно приобрести не подскажете ???\n'